In [1]:
from Auto3D.auto3D import options, main

/opt/anaconda3/envs/auto3D/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/auto3D/lib/python3.10/site-packages/Auto3D/isomer_engine.py:22: UserWarning: Openeye software is not available.
  warnings.warn("Openeye software is not available.")
/opt/anaconda3/envs/auto3D/lib/python3.10/site-packages/Auto3D/batch_opt/batchopt.py:9: UserWarning: TorchANI is not installed
  warnings.warn("TorchANI is not installed")
/opt/anaconda3/envs/auto3D/lib/python3.10/site-packages/Auto3D/batch_opt/batchopt.py:15: UserWarning: ANI2xt model is not available
  warnings.warn("ANI2xt model is not available")


In [6]:
if __name__ == "__main__":
    path = "/Users/liu5/Documents/Auto3D_pkg/example/files/smiles.smi"
    args = options(path, k=1, use_gpu=False)   #args specify the parameters for Auto3D 
    out = main(args)            #main acceps the parameters and run Auto3D
    print(out)

Checking input file...
	There are 4 SMILES in the input file /Users/liu5/Documents/Auto3D_pkg/example/files/smiles.smi. 
	All SMILES and IDs are valid.
Suggestions for choosing isomer_engine and optimizing_engine: 
	Isomer engine options: RDKit and Omega.
	Optimizing engine options: ANI2x, ANI_2xt and AIMNET.
There are 4 SMILES, available memory is 16.0 GB.
The task will be divided into 1 jobs.
Job1, number of inputs: 4


/opt/anaconda3/envs/auto3D/lib/python3.10/site-packages/Auto3D/isomer_engine.py:22: UserWarning: Openeye software is not available.
  warnings.warn("Openeye software is not available.")
/opt/anaconda3/envs/auto3D/lib/python3.10/site-packages/Auto3D/isomer_engine.py:22: UserWarning: Openeye software is not available.
  warnings.warn("Openeye software is not available.")
/opt/anaconda3/envs/auto3D/lib/python3.10/site-packages/Auto3D/batch_opt/batchopt.py:9: UserWarning: TorchANI is not installed
  warnings.warn("TorchANI is not installed")
/opt/anaconda3/envs/auto3D/lib/python3.10/site-packages/Auto3D/batch_opt/batchopt.py:9: UserWarning: TorchANI is not installed
  warnings.warn("TorchANI is not installed")
/opt/anaconda3/envs/auto3D/lib/python3.10/site-packages/Auto3D/batch_opt/batchopt.py:15: UserWarning: ANI2xt model is not available
  warnings.warn("ANI2xt model is not available")
/opt/anaconda3/envs/auto3D/lib/python3.10/site-packages/Auto3D/batch_opt/batchopt.py:15: UserWarning: A



Isomer generation for job1
Enumerating cis/tran isomers for unspecified double bonds...
Enumerating R/S isomers for unspecified atomic centers...
Removing enantiomers...
Enumerating conformers/rotamers, removing duplicates...


Optimizing on job1
Preparing for parallel optimizing... (Max optimization steps: 10000)
Total 3D conformers: 54


 37%|███▋      | 3682/10000 [02:16<03:53, 27.05it/s] 


Total 3D structures: 54  Converged: 38   Active: 16
Total 3D structures: 54  Converged: 53   Active: 1
Total 3D structures: 54  Converged: 53   Active: 1
All structures converged at step 3683:   Total 3D structures: 54  Converged: 54   Active: 0
Beggin to slelect structures that satisfy the requirements...
Energy unit: Hartree if implicit.
Program running time: 140.73682284355164 seconds
/Users/liu5/Documents/Auto3D_pkg/example/files/20220428-173956_smiles/smiles_out.sdf


In [2]:
help(options)

Help on function options in module Auto3D.auto3D:

options(path, k=False, window=False, verbose=False, job_name='', enumerate_tautomer=False, tauto_engine='rdkit', isomer_engine='rdkit', cis_trans=True, mode_oe='classic', mpi_np=4, max_confs=1000, use_gpu=True, gpu_idx=0, capacity=42, optimizing_engine='AIMNET', opt_steps=10000, convergence_threshold=0.003, threshold=0.3)
    Arguments for Auto3D main program
    path: A input.smi containing SMILES and IDs. Examples are listed in the input folder
    k: Outputs the top-k structures for each SMILES.
    window: Outputs the structures whose energies are within window (kcal/mol) from the lowest energy
    verbose: When True, save all meta data while running.
    job_name: A folder name to save all meta data.
    
    enumerate_tautomer: When True, enumerate tautomers for the input
    taut_engine: Programs to enumerate tautomers, either 'rdkit' or 'oechem
    isomer_engine: The program for generating 3D isomers for each SMILES. This param

In [3]:
help(main)

Help on function main in module Auto3D.auto3D:

main(args: dict)
    Take the arguments from options and run Auto3D

